In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from funcs.distance import get_all_distances, get_distance_matrix
from funcs.convert import convert_to_picometers, get_bond_type, get_bond_counts

from ase.db import connect

import numpy as np
import pandas as pd

In [52]:
train_db = connect("./data/train.db")
row = train_db.get(13)
n = row.natoms
p = row.positions
s = row.symbols

In [53]:
distance_matrix = get_distance_matrix(row)
obj = {
    'Br':1, 
    'C':4, 
    'Cl':1, 
    'F':1, 
    'H':1, 
    'N':3, 
    'O':2, 
    'S':2
}
shortest_bond_length = 80
longest_bond_length = 187
type_bond_length =  {'BrC': [191], 'BrN': [188], 'BrO': [180], 'BrS': [218], 'CC': [134, 154, 140], 'CCl': [176], 'CF': [141], 'CH': [114], 'CN': [132, 151], 'CO': [124, 143], 'CS': [162, 181], 'ClN': [173], 'ClO': [165], 'ClS': [203], 'FN': [138], 'FO': [130], 'FS': [168], 'HN': [111], 'HO': [103], 'HS': [141], 'NN': [130, 148], 'NO': [122, 140], 'NS': [160, 178], 'OO': [132], 'OS': [170], 'SS': [208]}

In [54]:
def get_bond_counts_limiter(distance_matrix, symbols):
    n_atoms = len(symbols)
    
    distance_matrix = convert_to_picometers(distance_matrix)
    bond_counter = {}
    available_bonds = [obj[atom] for atom in s]
    for i in range(n_atoms - 1):
        for j in range(i, n_atoms):
            bond_name = "".join(sorted(symbols[i] + symbols[j]))
            bond_length = distance_matrix[i][j]
            
            if(bond_length < shortest_bond_length or 
                bond_length > longest_bond_length or
                bond_name not in type_bond_length or 
                available_bonds[i] <= 0 or available_bonds[j] <= 0):
                continue
            else:
                available_bonds[i] -= 1
                available_bonds[j] -= 1
                bond_type = get_bond_type(bond_name, bond_length)
                bond_name += str(bond_type)
                
                if bond_name in bond_counter:
                    bond_counter[bond_name] += 1
                else:
                    bond_counter[bond_name] = 1

    return bond_counter
counter = get_bond_counts_limiter(distance_matrix, s)
print(counter)
counter1 = get_bond_counts(distance_matrix, s)
print(counter1)

{'CC1': 1, 'CH1': 17, 'CC2': 4, 'CN2': 2, 'CN1': 8, 'CS2': 2, 'CC1.5': 2, 'CO1': 2, 'HN1': 1}
{'CC1': 1, 'CH1': 17, 'CC2': 4, 'CN2': 2, 'CN1': 8, 'CS2': 2, 'CC1.5': 2, 'CO1': 2, 'HN1': 1}
